In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from sklearn.model_selection import train_test_split
import os

In [2]:
# Set random seed for reproducibility
tf.random.set_seed(42)


In [3]:
# Paths to datasets
data_dir = "C:/Users/5A_Traders/Desktop/GTM Data"
image_size = (224, 224)  # MobileNetV2 input size
batch_size = 32
epochs = 10
learning_rate = 0.0001

In [4]:
# Data Preparation
datagen = ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.2,
    horizontal_flip=True,
    rotation_range=20,
    zoom_range=0.3,
    width_shift_range=0.2,
    height_shift_range=0.2
)

In [5]:
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    subset="training",
    class_mode="categorical"
)

val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    subset="validation",
    class_mode="categorical"
)

Found 1203 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


In [6]:
print(train_generator.labels.shape)

(1203,)


In [7]:
class_indices = train_generator.class_indices
class_labels = list(class_indices.keys())

# Compute class weights to handle class imbalance
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.arange(len(class_labels)),  # Class labels as integers
    y=train_generator.labels  # Use labels directly
)

# Convert to a dictionary to map class indices to weights
class_weights = dict(enumerate(class_weights))

print(class_weights) 

{0: 1.0100755667506298, 1: 0.9901234567901235, 2: 1.0}


In [8]:
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Fine-tuning: Unfreeze the last few layers of the base model
for layer in base_model.layers[-30:]:
    layer.trainable = True

In [9]:
# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(64, activation="relu")(x)
x = Dropout(0.3)(x)
predictions = Dense(train_generator.num_classes, activation="softmax")(x)

In [10]:
model = Model(inputs=base_model.input, outputs=predictions)


In [11]:
# Compile the model with gradient clipping
opt = Adam(learning_rate=learning_rate, clipnorm=1.0)
model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])

In [12]:
# Early stopping and model checkpoint
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=5, restore_best_weights=True
    ),
    tf.keras.callbacks.ModelCheckpoint(
        "best_model.keras", monitor="val_loss", save_best_only=True
    ),
]

In [13]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_steps=val_generator.samples // batch_size,
    class_weight=class_weights,
    callbacks=callbacks,
    verbose=1,
)

Epoch 1/10


C:\Users\5A_Traders\miniconda3\envs\GTM\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


37/37 ━━━━━━━━━━━━━━━━━━━━ 225s 4s/step - accuracy: 0.6088 - loss: 0.8507 - val_accuracy: 0.9688 - val_loss: 0.1135
Epoch 2/10
 1/37 ━━━━━━━━━━━━━━━━━━━━ 2:05 3s/step - accuracy: 0.8438 - loss: 0.2595

C:\Users\5A_Traders\miniconda3\envs\GTM\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


37/37 ━━━━━━━━━━━━━━━━━━━━ 5s 55ms/step - accuracy: 0.8438 - loss: 0.2595 - val_accuracy: 1.0000 - val_loss: 0.0065
Epoch 3/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 147s 4s/step - accuracy: 0.9494 - loss: 0.1466 - val_accuracy: 0.9653 - val_loss: 0.0800
Epoch 4/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9688 - loss: 0.0789 - val_accuracy: 0.9167 - val_loss: 0.2928
Epoch 5/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 152s 4s/step - accuracy: 0.9712 - loss: 0.0778 - val_accuracy: 0.9722 - val_loss: 0.0639
Epoch 6/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9688 - loss: 0.0581 - val_accuracy: 1.0000 - val_loss: 0.0171
Epoch 7/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 150s 4s/step - accuracy: 0.9778 - loss: 0.0677 - val_accuracy: 0.9653 - val_loss: 0.1163


In [14]:
# Save the final model
model.save("mobilenetv2_few_shot_model.h5")

In [15]:
# Save the model
model.save("mobilenetv2_image_classifier.keras")

In [18]:

# Prediction Function for Few-Shot Learning
def predict_image(image_path):
    from tensorflow.keras.preprocessing import image
    img = image.load_img(image_path, target_size=image_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions)
    confidence = np.max(predictions)
    return train_generator.class_indices, predicted_class, confidence

In [20]:
# Example Usage
class_indices, predicted_class, confidence = predict_image("C:/Users/5A_Traders/Desktop/TestData/pexels-suju-1271816.jpg")
print("Class Indices:", class_indices)
print("Predicted Class:", predicted_class)
print("Confidence:", confidence)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
Class Indices: {'cats': 0, 'dogs': 1, 'mountain': 2}
Predicted Class: 2
Confidence: 0.8341597
